In [257]:
import pandas as pd
import numpy as np
import os

In [258]:
start_y=2014
files=['tp20151211-01_1.xls',
'tp20151211-01_3.xls',
'tp20151211-01_1.xls',
'tp20151211-01_3.xls',
'tp20180314-01_1.xls',
'tp20171208-01_3.xls',
'tp20180314-01_1.xls',
'tp20171208-01_3.xls',
'tp20190904-01_01.xls',
'tp20190904-01_03.xls',
'tp20200122-01_01.xls',
'tp20200122-01_03.xls',
'tp20210218-01_01.xlsx',
'tp20210218-01_03.xlsx',
'tp20220201-01_01.xlsx',
'tp20211210-01_03.xlsx']
for i,name in enumerate(files):
    print(start_y+i//2,name)

2014 tp20151211-01_1.xls
2014 tp20151211-01_3.xls
2015 tp20151211-01_1.xls
2015 tp20151211-01_3.xls
2016 tp20180314-01_1.xls
2016 tp20171208-01_3.xls
2017 tp20180314-01_1.xls
2017 tp20171208-01_3.xls
2018 tp20190904-01_01.xls
2018 tp20190904-01_03.xls
2019 tp20200122-01_01.xls
2019 tp20200122-01_03.xls
2020 tp20210218-01_01.xlsx
2020 tp20210218-01_03.xlsx
2021 tp20220201-01_01.xlsx
2021 tp20211210-01_03.xlsx


In [259]:
#ディレクトリ直下のファイルのパスを取得する
firm_data=pd.DataFrame()
for i,name in enumerate(files):
    print(start_y+i//2,name)
    data_i=pd.read_excel(name)
    data_i['year']=start_y+i//2
    firm_data = pd.concat([firm_data,data_i],axis=0,ignore_index=True)
    this_year = start_y+i//2
    last_year = (start_y+i//2)-1
    if i%2==1:
        # 現在のDataFrameから年とIDの組み合わせを取得
        firm_data_two=firm_data[firm_data['year']>=(last_year)]
        existing_combinations = set(zip(firm_data_two['year'], firm_data_two["薬価基準収載医薬品コード"]))

        # 昨年に対応する行を作成し、今年のIDがない場合、今年のIDを持つ行をコピー
        new_rows = []
        for this_year, id in existing_combinations:
            if (last_year, id) not in existing_combinations:
                this_year_row = firm_data[(firm_data['year'] == this_year) & (firm_data["薬価基準収載医薬品コード"] == id)]
                new_row = this_year_row.copy()
                new_row['year'] = last_year
                new_rows.append(new_row)
        if this_year==2019 or this_year==2020:
            # this yearに対応する行を作成し、last yearのIDがない場合、last yearのIDを持つ行をコピー
            for last_year, id in existing_combinations:
                if (this_year, id) not in existing_combinations:
                    last_year_row = firm_data[(firm_data['year'] == last_year) & (firm_data["薬価基準収載医薬品コード"] == id)]
                    new_row = last_year_row.copy()
                    new_row['year'] = this_year
                    new_rows.append(new_row)
        # 新しい行を現在のDataFrameに追加
        firm_data = pd.concat([firm_data] + new_rows,axis=0, ignore_index=True)
    

2014 tp20151211-01_1.xls
2014 tp20151211-01_3.xls
2015 tp20151211-01_1.xls
2015 tp20151211-01_3.xls
2016 tp20180314-01_1.xls
2016 tp20171208-01_3.xls
2017 tp20180314-01_1.xls
2017 tp20171208-01_3.xls
2018 tp20190904-01_01.xls
2018 tp20190904-01_03.xls
2019 tp20200122-01_01.xls
2019 tp20200122-01_03.xls
2020 tp20210218-01_01.xlsx
2020 tp20210218-01_03.xlsx
2021 tp20220201-01_01.xlsx
2021 tp20211210-01_03.xlsx


In [260]:
firm_data=firm_data[firm_data["メーカー名"].notna()]
firm_data.shape

(111256, 21)

大正薬品工業は名前を変えた

In [261]:
firm_data["year"]

1         2014
2         2014
3         2014
4         2014
5         2014
          ... 
121777    2020
121778    2020
121779    2020
121780    2020
121781    2020
Name: year, Length: 111256, dtype: int64

In [262]:
dup=firm_data[firm_data.duplicated(["薬価基準収載医薬品コード","year"])]
#同じ年で名前が変わっている? 2023まで入れればね
for id in set(dup["薬価基準収載医薬品コード"]):
    df_id=dup[dup["薬価基準収載医薬品コード"]==id]
    year=set(df_id["year"])
    maker=set(df_id["メーカー名"])
    if len(maker)>1:
        print(id,year)
        print(maker)

In [263]:
len(set(firm_data["メーカー名"])),len(set(firm_data["薬価基準収載医薬品コード"])),sum(firm_data["メーカー名"].isna())

(337, 16386, 0)

In [264]:
print(set(firm_data["メーカー名"]))

{'ファイザー', 'メルクバイオファーマ', '生化学工業', '共和薬品工業', '小西製薬', 'アストラゼネカ', 'ユーシービージャパン', 'レクメド', '日新製薬（山形）', '生晃栄養薬品', 'アルフレッサファーマ', 'ヤクルト本社', 'アミカス・セラピューティクス', 'キッセイ薬品工業', 'クラシエ製薬', '興和', '日医工', '堀井薬品工業', 'クリニジェン', '陽進堂', 'ファイザーＵＰＪ', '鈴粉末薬品', 'ジェーピーエス製薬', 'アボットジャパン', '第一三共', 'グラフィコ', '帝國漢方製薬', '大塚製薬工場', '旭化成ファーマ', '日東電工', '富士製薬工業', '大杉製薬', 'サンケミファ', 'ヴィアトリス製薬', 'シオノギファーマ', '日本たばこ産業', 'フヂミ製薬所', 'ヴィーブヘルスケア', 'ＬＴＬファーマ', '富士フイルムワコーケミカル', 'アサヒグループ食品', '味の素製薬', 'ビオメディクス', 'アステラス製薬', '健栄製薬', 'ブリストル・マイヤーズ', '日本点眼薬研究所', '皇漢堂製薬', 'ニプロＥＳファーマ', '薬師製薬', 'グラクソ・スミスクライン・コンシューマー・ヘルスケア・ジャパン', '日本ベーリンガーインゲルハイム', '東興薬品工業', 'ケミックス', 'スキャンポファーマ', '武田テバ薬品', 'マイラン製薬', 'シャイアー・ジャパン', '小林化工', 'ローマン工業', '高田製薬', 'ニプロパッチ', '沢井製薬', '小野薬品工業', '協和発酵キリン', 'エフピー', 'バイオジェン・ジャパン', '康和薬通', 'レオファーマ', '昭和製薬', '富士化学工業', 'シー・エイチ・オー新薬', 'ミヤリサン製薬', 'アスペンジャパン', '高杉製薬', 'ＫＭバイオロジクス', 'ヤクハン製薬', '佐藤製薬', '同仁医薬化工', '鳥居薬品', '原沢製薬工業', '日本化薬', '小堺製薬', '日本ビーシージー製造', '東菱薬品工業', '鶴原製薬', '昭和薬品化工', '今津薬品工業', 'Ｈｕｙａ\u3000Ｊａｐａｎ', '日本ジェネリック', '日東薬品工業', '日興製薬',

In [265]:
data_ndb=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
# data_n=data_n[data_n["year"]>=2018]
print("initial",len(set(data_ndb["薬価基準収載医薬品コード"])))

initial 2671


In [266]:
data_ndb.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital',
       ...
       'id_625.0', 'id_629.0', 'id_799.0', 'q_share', 'r_share',
       'generic_share_q', 'generic_share_r', 'lag_generic_share_q',
       'mean_price_g', 'mean_price_b'],
      dtype='object', length=112)

In [267]:
len(set(data_ndb["薬価基準収載医薬品コード"]) - set(firm_data["薬価基準収載医薬品コード"]))

207

In [268]:
data_ndb.shape

(25798, 112)

In [269]:
data_ndb.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital',
       ...
       'id_625.0', 'id_629.0', 'id_799.0', 'q_share', 'r_share',
       'generic_share_q', 'generic_share_r', 'lag_generic_share_q',
       'mean_price_g', 'mean_price_b'],
      dtype='object', length=112)

In [270]:
data_ndb.duplicated(subset=['year', "医薬品コード","in_hospital"],keep=False).sum()

0

yearを最初は入れない

In [271]:
# data_tmp=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード"],how="left",indicator=True)
# condition = (data_tmp['year_x'] != data_tmp['year_y']) & data_tmp.duplicated(subset=['year_x', "医薬品コード","in_hospital"], keep=False)
# # condition =  data_tmp.duplicated(subset=['year_x', "医薬品コード","in_hospital"],keep=False)
# data_tmp = data_tmp[~condition]
# data_left_only=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード"],how="left",indicator=True).query('_merge == "left_only"')
# print(data_tmp.shape,data_left_only.shape)
# print(data_tmp["_merge"].value_counts())

(25701, 133) (3739, 133)
both          21962
left_only      3739
right_only        0
Name: _merge, dtype: int64


In [272]:
# duplicated かつ一つもyearが一致しないものがある?
# condition =  data_tmp.duplicated(subset=['year_x', "医薬品コード","in_hospital"],keep=False)
# data_tmp = data_tmp[condition]
# data_tmp[["in_hospital","医薬品コード","year_x"]].value_counts()
# data_tmp2=data_tmp[~(data_tmp['year_x'] != data_tmp['year_y'])]
# data_tmp2[["in_hospital","医薬品コード","year_x"]].value_counts()
# set(map(tuple,data_tmp[["in_hospital","医薬品名","year_x"]].to_records(index=False)))-set(map(tuple,data_tmp2[["in_hospital","医薬品名","year_x"]].to_records(index=False)))
# len(set(data_left_only["薬価基準収載医薬品コード"]))

year を入れてもうまくいく

In [273]:
data_tmp=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード","year"],how="left",indicator=True)
data_left_only=data_ndb.merge(firm_data,on=["薬価基準収載医薬品コード","year"],how="left",indicator=True).query('_merge == "left_only"')
print(data_tmp.shape,data_left_only.shape)
print(data_tmp["_merge"].value_counts())

(25798, 132) (3836, 132)
both          21962
left_only      3836
right_only        0
Name: _merge, dtype: int64


In [274]:
dups=data_tmp[data_tmp.duplicated(subset=["year","医薬品コード","in_hospital"],keep=False)]
print(dups[["year","医薬品コード","メーカー名","in_hospital"]])

Empty DataFrame
Columns: [year, 医薬品コード, メーカー名, in_hospital]
Index: []


In [275]:
len(set(data_left_only["薬価基準収載医薬品コード"])-set(firm_data["薬価基準収載医薬品コード"]))

207